In [ ]:
from pyspark.sql import SparkSession
import collections

In [ ]:
spark = SparkSession.builder.appName("logAnalysis").getOrCreate()

In [44]:
lines = spark.readStream.text("data")
lines.isStreaming

TypeError: text() got an unexpected keyword argument 'maxFileAge'

In [38]:
regex = "^[0-9]{1,2}\\/[0-9]{1,2}\\/[0-9]{2} [0-9]{1,2}\:[0-9]{1,2}\:[0-9]{2} [A-Z]{4,5} "
lines = lines.filter(lines["value"].rlike(regex))

In [39]:
from pyspark.sql.functions import split, explode
df_with_log_and_class = lines.withColumn("level", split(lines['value'], ' ').getItem(2)) \
                        .withColumn('className', split(lines["value"], " ").getItem(3)) \
                        .withColumn('date', split(lines["value"], " ").getItem(0)) \
                        .drop('value')
# df_with_log_and_class.show(2)

In [40]:
df_with_log_and_class.printSchema()

root
 |-- level: string (nullable = true)
 |-- className: string (nullable = true)
 |-- date: string (nullable = true)



In [41]:
df_with_log_and_class.createOrReplaceTempView("df1_view")
# result = spark.sql("select level,className,count(level) from df1_view   group by className, level order by count(level) ")
result = spark.sql("select * from df1_view")

In [ ]:
result.writeStream.format("console").outputMode("append").start().awaitTermination()

23/01/17 12:44:00 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-23832275-6f46-432a-aeae-f14361940d29. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/17 12:44:00 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
-------------------------------------------
Batch: 0
-------------------------------------------
+-----+--------------------+--------+
|level|           className|    date|
+-----+--------------------+--------+
| INFO|executor.CoarseGr...|17/06/09|
| INFO|spark.SecurityMan...|17/06/09|
| INFO|spark.SecurityMan...|17/06/09|
| INFO|spark.SecurityMan...|17/06/09|
| INFO|    saransh.saransh:|17/06/09|
| INFO|    saransh.saransh:|17/06/09|
| INFO|    saransh.saransh:|17/0